In [1]:
# Importe de librerias que se van a utilizar
import time
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import TensorBoard

# Configuracion inicial de la red neuronal
img_width, img_height = 70,70 # width y height para las imagen de entrada
input_depth = 1 #1: Transformación a escala de grises
train_data_dir = '../datasets/ChairsDataset/training' # Direccion de la carpeta de training
testing_data_dir = '../datasets/ChairsDataset/testing' # Direccion de la carpeta de testing
epochs = 10 # Numero de epochs a utilizar
batch_size = 5 # Numero de Batches a utilizar

# Definicion de la generacion de imagenes para Keras,
# Nomalizacion de las imagenes de entrada, (valores de 0 a 1)
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

# Lectura de las imagenes por batch
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='grayscale',
    target_size=(img_width,img_height),# Tamano de las imagenes de entrada
    batch_size=batch_size,# Tamaño batch
    shuffle=True,# aleatorizacion de los datos
    class_mode='categorical')# Definición del modelo de red neuronal como categorico
testing_generator = test_datagen.flow_from_directory(
    testing_data_dir,
    color_mode='grayscale',
    target_size=(img_width,img_height),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical')

# Confirmacion de la configuracion de la red neuronal
print(train_generator.class_indices)






print(testing_generator.class_indices)

Found 2500 images belonging to 5 classes.
Found 500 images belonging to 5 classes.
{'Comedor': 0, 'Gamer': 1, 'Mesedoras': 2, 'Taburete': 3, 'Terraza': 4}
{'Comedor': 0, 'Gamer': 1, 'Mesedoras': 2, 'Taburete': 3, 'Terraza': 4}


In [2]:
# Definición de la forma de la imagen de entrada
if K.image_data_format() == 'channels_first':
    input_shape_val = (input_depth, img_width, img_height)
else:
    input_shape_val = (img_width, img_height, input_depth)

NAME = "RN_sillas-{}".format(int(time.time()))

# Definicon de la red
model = Sequential()

# Primera capa convolucional
model.add(Conv2D(64, (5, 5), 
                 input_shape=input_shape_val, 
                 padding='same', name='input_tensor'))
model.add(Activation('relu'))
model.add(MaxPool2D((2, 2)))

# Segunda capa convolucional
model.add(Conv2D(64, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D((2, 2)))

# Tercera capa convolucional
model.add(Conv2D(64, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D((2, 2)))

# Aplanamiento de las imagenes
model.add(Flatten())

# Capa de salida
model.add(Dense(train_generator.num_classes, 
                activation='softmax', name='output_tensor'))

# Direccion de la carpeta donde se guardaran los logs (TensorBoard)
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

# Compilación de la red
model.compile(loss='categorical_crossentropy',
              optimizer='sgd', metrics=['accuracy'])

# Resumen del modelo de la red
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_tensor (Conv2D)       (None, 70, 70, 64)        1664      
                                                                 
 activation (Activation)     (None, 70, 70, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 35, 35, 64)       0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 35, 35, 64)        102464    
                                                                 
 activation_1 (Activation)   (None, 35, 35, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 17, 17, 64)       0         
 2D)                                                    

In [3]:
# Entrnemaiento y testeo de la red
model.fit(
    train_generator,#Nuentro generador de entrenamiento
    # Numero de iteraciones por epoch = numero de datos / tamaño batch
    steps_per_epoch=np.floor(train_generator.n/batch_size),
    epochs=epochs,# Numero de epochs
    validation_data=testing_generator,#our validation generator Nuentro generador de alidación 
    # Numero de iteraciones por epoch = numero de datos / tamaño batch
    validation_steps=np.floor(testing_generator.n / batch_size),
    callbacks=[tensorboard])# Registro de los logs en TensorBoard

Epoch 1/10
500/500 [==============================] - 36s 71ms/step - loss: 1.3699 - accuracy: 0.4144 - val_loss: 0.8648 - val_accuracy: 0.6740
Epoch 2/10
500/500 [==============================] - 35s 69ms/step - loss: 0.7768 - accuracy: 0.7136 - val_loss: 0.6221 - val_accuracy: 0.7940
Epoch 3/10
500/500 [==============================] - 35s 70ms/step - loss: 0.5622 - accuracy: 0.8084 - val_loss: 0.5051 - val_accuracy: 0.8220
Epoch 4/10
500/500 [==============================] - 35s 70ms/step - loss: 0.4357 - accuracy: 0.8580 - val_loss: 0.4346 - val_accuracy: 0.8460
Epoch 5/10
500/500 [==============================] - 35s 69ms/step - loss: 0.3518 - accuracy: 0.8796 - val_loss: 0.4088 - val_accuracy: 0.8700
Epoch 6/10
500/500 [==============================] - 33s 66ms/step - loss: 0.2969 - accuracy: 0.9028 - val_loss: 0.4222 - val_accuracy: 0.8640
Epoch 7/10
500/500 [==============================] - 34s 67ms/step - loss: 0.2313 - accuracy: 0.9220 - val_loss: 0.4428 - val_accuracy:

In [4]:
print("Entrenamiento terminado!")
model.save('./model/{}.h5'.format(NAME))
print("El modelo se ha guardado con éxito en: ./model/{}.h5".format(NAME))

Entrenamiento terminado!
El modelo se ha guardado con éxito en: ./model/RN_sillas-1646158450.h5
